In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/LG_Bootcamp/SD'

/content/drive/MyDrive/LG_Bootcamp/SD


### Install packages

In [3]:
!pip install --upgrade diffusers[torch] -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.8 MB/s eta 0:00:00


In [8]:
!pip install huggingface_hub -q

### Download Model weights

In [3]:
from pathlib import Path

def ensure_dir(dirname):
    dirname = Path(dirname)
    if not dirname.is_dir():
        dirname.mkdir(parents=True, exist_ok=False)

In [4]:
model_path = '/content/drive/MyDrive/LG_Bootcamp/SD/weights/SDv2-1'
LoRA_path = '/content/drive/MyDrive/LG_Bootcamp/SD/weights/'
output_path =  '/content/drive/MyDrive/LG_Bootcamp/SD/output'

ensure_dir(model_path)
ensure_dir(LoRA_path)
ensure_dir(output_path)

In [13]:
from diffusers import StableDiffusionPipeline
import torch

model_name = "stabilityai/stable-diffusion-2-1"

pipe = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16  # VRAM 절약
)

# save_model
pipe.save_pretrained(model_path)
print(f"모델이 {model_path} 폴더에 저장되었습니다.")

model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/939 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

모델이 /content/drive/MyDrive/LG_Bootcamp/SD/weights/SD 폴더에 저장되었습니다.


In [34]:
from huggingface_hub import hf_hub_download

lora_repo = "lora-library/B-LoRA-watercolor"
lora_filename = "pytorch_lora_weights.safetensors"
lora_name = "watercolor"

save_path = os.path.join(LoRA_path, lora_name)
ensure_dir(save_path)

LoRA_path = hf_hub_download(
    repo_id=lora_repo,
    filename=lora_filename,
    local_dir=save_path
)

print(f"✅ LoRA 가중치 다운로드 완료! 저장 경로: {LoRA_path}")

pytorch_lora_weights.safetensors:   0%|          | 0.00/113M [00:00<?, ?B/s]

✅ LoRA 가중치 다운로드 완료! 저장 경로: /content/drive/MyDrive/LG_Bootcamp/SD/weights/pen_sketch/pytorch_lora_weights.safetensors


### Inference

In [5]:
import os
import gc
import torch
from diffusers import DiffusionPipeline, StableDiffusionPipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [6]:
seed = 42
generator = torch.Generator("cuda").manual_seed(seed)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [7]:
gc.collect()
torch.cuda.empty_cache()

In [19]:
prompt_method = """
Rainy day, warm tea by the window, soft candlelight. A book open, raindrops blur the view.
Faint childhood memory of rain boots splashing. {}. rich brushstrokes, nostalgic serenity."""

prompt = """
Rainy day, warm tea by the window, soft candlelight. A book open, raindrops blur the view.
Faint childhood memory of rain boots splashing. rich brushstrokes, nostalgic serenity."""

In [23]:
prompt_illustration = """
A vibrant anime-style city at sunset, cherry blossom petals floating in the wind.
A young protagonist in a school uniform, standing on a bridge, looking at the glowing skyline.
(sharp cel shading:1.3), (Studio Ghibli style:1.4), (webtoon aesthetic:1.3), (soft gradient lighting:1.2).
"""

prompt_painting = """
A rustic countryside landscape, painted with thick oil brushstrokes, warm golden sunset over rolling hills.
A small wooden cottage with smoke rising from the chimney, surrounded by vibrant flowers.
(van Gogh style:1.4), (rich texture:1.3), (bold impressionist strokes:1.3), (soft blended colors:1.2).
"""

prompt_photorealistic = """
A breathtaking mountain lake at sunrise, crystal-clear reflections on the water.
A hiker in a red jacket, standing on a rocky ledge, admiring the vast landscape.
(ultra-detailed:1.3), (HDR lighting:1.4), (cinematic wide-angle shot:1.3), (National Geographic photography:1.4).
"""

prompt_3D = """
A futuristic cyberpunk city, glowing neon signs reflecting on the rain-soaked streets.
A sleek humanoid robot walking through the bustling market, detailed CGI textures and dynamic lighting.
(Unreal Engine render:1.4), (Pixar style shading:1.3), (high-poly CGI:1.4), (soft depth of field:1.2).
"""

In [17]:
pipe = StableDiffusionPipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16  # VRAM 절약
).to("cuda")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# - 일러스트 `anime, manga, cartoon, webtoon`
# - 페인팅 `oil painting, watercolor, impressionist`
# - 사진 `photorealistic, cinematic, HDR, high contrast`
# - 3D `CGI, voxel art, Unreal Engine, Pixar style`

In [ ]:
# # 원본 SD inference
# style_info = 'anime'

# image = pipe(prompt_method.replace('{}', 'voxel art'), generator=generator).images[0]
# image.save(f"{output_path}/{style_info}.png")

In [24]:
styles = {
    'illustration' : prompt_illustration,
    'painting' : prompt_painting,
    'realistic' : prompt_photorealistic,
    '3D' : prompt_3D,


}
for name, _prompt in styles.items():
    image = pipe(_prompt, generator=generator).images[0]
    image.save(f"{output_path}/SD_{name}.png")

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# LoRA 가중치 로드
lora_name = "LoRA"
pipe.load_lora_weights(os.path.join(LoRA_path, lora_name) )

image = pipe(prompt, generator=generator).images[0]
image.save(f"{output_path}/SD_LoRA.png")

In [30]:
# LoRA 가중치 제거
pipe.unload_lora_weights()

image_original_again = pipe(prompt_method.replace('{}', 'voxel art'), generator=generator).images[0]
image_original_again.save(f"{output_path}/SD_original_again.png")

  0%|          | 0/50 [00:00<?, ?it/s]

### LoRA training

In [31]:
# https://huggingface.co/blog/lora